In [2]:
import pandas as pd
import glob
import os
import numpy as np
import lda
import lda.datasets

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [3]:
dataset = []

path = 'datas/Transcript/'
allFiles = glob.glob(os.path.join(path, '*.csv'))

for file_ in allFiles:
    data = pd.read_csv(file_)
    dataset = dataset + list(data[(data['Text'] == data['Text']) & (data['LanguageOfText'] == 'FR')]['Text'].values)
    
print(len(dataset))
dataset[0]

27405


'Aujourd\'hui, nous connaissons une distinction entre la fraude fiscale et la soustraction fiscale. La fraude fiscale conduit en général à l\'utilisation de faux dans les titres; elle tombe sous le coup du droit pénal, avec des procédures judiciaires; elle aboutit à des peines sous forme d\'amendes ou à des peines privatives de liberté. La soustraction fiscale, quant à elle, relève de la négligence; elle est soumise à une procédure administrative et elle aboutit à une contravention, à une amende.\nL\'auteur de l\'initiative nous propose de faire une distinction à l\'intérieur du concept de soustraction fiscale, soit entre la soustraction intentionnelle ou répétée et la soustraction par négligence ou unique. Avec la soustraction intentionnelle, nous serions en présence d\'une fraude fiscale, c\'est-à-dire que nous aurions affaire à un délit ou à un crime, avec une procédure judiciaire à la clé. La notion de "soustraction intentionnelle" serait en relation avec des montants importants. P

In [3]:
print(dataset[301])

Je commencerai par éclaircir un point, au sujet de la proposition de la minorité Moret et de celle de la majorité, qui a donné lieu à des interprétations diverses. Il s'agit, concernant les médicaments qui bénéficient d'une autorisation cantonale, de savoir s'il y a une différence matérielle entre le fait d'inscrire cela à l'article 95b ou à l'article 9 alinéa 2 lettre f. En réalité, par rapport à la situation de départ aujourd'hui, et qui évoluerait en application de l'une ou l'autre disposition, il n'y a matériellement aucune différence. C'est la raison pour laquelle je vous laisse libres de décider. Cela ne change rien quant au contenu.
L'autre élément sur lequel je souhaite revenir concerne la question de l'octroi ou de la prolongation de l'exclusivité des données et la question de l'exclusivité commerciale. En complément à l'exclusivité des données, en général d'une durée de dix ans, comme le prévoit l'article 11a, la révision prévoit à l'article 11b alinéa 1 une durée de protecti

In [4]:
n_samples = 2000
n_features = 1000
n_topics = 10
n_top_words = 20

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

data_samples = dataset

stop_words = ['de', 'il', 'et', 'je', 'nous', 'sur', 'par', 'der', 'die', 'und', 'le', 'la', 'les', 'qui', 'que', 'du', 'un', 'une',
             'dans', 'ne', 'on', 'au', 'ce', 'ans', 'vs', 'commission', 'se', 'leur', 'ont', 'si', 'aux', 'votre', 'qu']
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words=stop_words)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

Extracting tf features for LDA...
done in 14.508s.


In [45]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Fitting LDA models with tf features, n_samples=2000 and n_features=1000...
done in 138.340s.

Topics in LDA model:
Topic #0:
des pour en et assurance est plus cent maladie pas prestations coûts système taux cette sont personnes santé soins primes
Topic #1:
et des est en proposition article minorité majorité pas pour conseil loi cette vous alinéa donc droit ou être elle
Topic #2:
et en des suisse est pour pays avec européenne union pas notre politique accord conseil etats droits convention fédéral cette
Topic #3:
des et en fédéral conseil motion loi pour est été cette projet confédération sont fédérale être cantons rapport ces mesures
Topic #4:
des pour et francs en millions est cent fédéral confédération plus conseil budget dépenses été montant sont ces milliards crédit
Topic #5:
est pas et des en pour mais plus tout cette cela sont peut être même vous ou fait bien faire
Topic #6:
initiative conseil et des contre en cette projet parlementaire été est notre vous voix suite etats pas ell

In [36]:

    
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
sécurité service armée armes militaire médicaments police civile civil protection engagement paix défense services la en pour être obligation assurer
Topic #1:
nous la le et que est les des en qui pour du il groupe une un pas ce notre avons
Topic #2:
la et des les en suisse le est pays dans que qui un pour une du il sur avec au
Topic #3:
la le commission conseil et des en une par du que est les article qui il proposition minorité majorité sur
Topic #4:
que est il pas qui on le je la qu ce un une et ne vous en dans les des
Topic #5:
le la du fédéral conseil et des au en sur pour par les dans été un ce que rapport commission
Topic #6:
la les des pour et le en francs une est qui par cent un que millions plus du assurance ce
Topic #7:
la les des et le en une par initiative du dans un que sur pour est qui ne au cette
Topic #8:
la les des et le en dans que pour est qui une sur il un ce du par nous cantons
Topic #9:
vs der die la und des in art proposition du président conseil vote 

In [5]:
import logging, gensim, bz2

# remove common words and tokenize
stoplist = ['de', 'il', 'et', 'je', 'nous', 'sur', 'par', 'der', 'die', 'und', 'le', 'la', 'les', 'qui', 'que', 'du', 'un', 'une',
             'dans', 'ne', 'on', 'au', 'ce', 'ans', 'vs', 'commission', 'se', 'leur', 'ont', 'si', 'aux', 'votre', 'qu']

texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in dataset]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

In [6]:
from gensim import corpora, models

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]


In [8]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=1)

In [10]:
print(ldamodel.print_topics(num_topics=3, num_words=3))
ldamodel.print_topics()

[(0, '0.026*"[vs]" + 0.012*"in" + 0.012*"des"'), (7, '0.034*"des" + 0.029*"à" + 0.024*"millions"'), (5, '0.031*"des" + 0.030*"à" + 0.021*"en"')]


[(0,
  '0.026*"[vs]" + 0.012*"in" + 0.012*"des" + 0.009*"-" + 0.009*"den" + 0.008*"das" + 0.008*"zu" + 0.008*"für" + 0.008*"antrag" + 0.007*"wir"'),
 (1,
  '0.037*"à" + 0.022*"des" + 0.021*"conseil" + 0.020*"proposition" + 0.018*"minorité" + 0.017*"l\'article" + 0.015*"a" + 0.015*"en" + 0.011*"majorité" + 0.010*"vous"'),
 (2,
  '0.036*"des" + 0.026*"à" + 0.023*"pour" + 0.022*"en" + 0.011*"pas" + 0.011*"est" + 0.010*"plus" + 0.007*"sont" + 0.006*"a" + 0.006*"cette"'),
 (3,
  '0.028*"à" + 0.020*"pas" + 0.020*"en" + 0.019*"des" + 0.014*"a" + 0.013*"c\'est" + 0.012*"pour" + 0.011*"est" + 0.010*"mais" + 0.008*"-"'),
 (4,
  '0.037*"à" + 0.028*"des" + 0.017*"en" + 0.015*"pour" + 0.010*"est" + 0.010*"pas" + 0.009*"a" + 0.007*"vous" + 0.007*"notre" + 0.007*"plus"'),
 (5,
  '0.031*"des" + 0.030*"à" + 0.021*"en" + 0.017*"pour" + 0.010*"est" + 0.009*"a" + 0.008*"suisse" + 0.007*"conseil" + 0.007*"cette" + 0.007*"formation"'),
 (6,
  '0.031*"des" + 0.031*"à" + 0.027*"en" + 0.022*"pour" + 0.012*"a" 